# 75.06 / 95.58 Organización de Datos
## Trabajo Práctico 1: Análisis Exploratorio de Datos

**Grupo 30: Datatouille**

- 101055 - Bojman, Camila
- 100029 - del Mazo, Federico
- 100687 - Hortas, Cecilia
- 97649 - Souto, Rodrigo

**https://github.com/fdelmazo/7506-datos**

### Introducción

Presentamos aca un análisis de datos obtenidos de de usuarios que visitaron www.trocafone.com, un sitio de e-commerce de compra y venta de celulares reacondicionados, con operaciones principalmente en Brasil.

In [ ]:
# Set-Up inicial

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import squarify # https://github.com/laserson/squarify ( pip install squarify )
import geopandas as gpd # http://geopandas.org/install.html ( conda install -c conda-forge geopandas )
from shapely.geometry import Point
from wordcloud import WordCloud # pip install wordcloud
from time import strptime
import calendar
%matplotlib inline

df = pd.read_csv('data/events.csv', low_memory=False, index_col='timestamp')

In [ ]:
sns.set(style="darkgrid")
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['figure.figsize'] = (15,5)

## Información general del dataset

In [ ]:
with pd.option_context('display.max_column',0):
    display(df.sample(n=5))

In [ ]:
bytes_used = df.memory_usage().sum()
print('Memoria usada: {:.2f}MB'.format(bytes_used/1000000))
print('{} atributos y {} registros en el dataframe.\n'.format(df.shape[1],df.shape[0]))
print('Primer registro: {} \nÚltimo registro: {}.'.format(df.index.min(),df.index.max()))

In [ ]:
describe = df.describe().T
descripcion = pd.read_csv('data/columns-desc.csv',index_col='column')
data = pd.merge(descripcion,describe,left_index=True,right_index=True)
data['null count'] = df.isnull().sum()
data['dtype'] = df.dtypes
with pd.option_context('display.max_colwidth',-1):
    display(data)

**Pasado el vistazo general a los datos, se pueden transformar datos en sus tipos correspondientes para mejor manejo de estos y para ahorro de memoria**

In [ ]:
# Los atributos con pocos valores posibles se pasan a variables categoricas para ahorrar memoria
df['event'] = df['event'].astype('category')
df['condition'] = df['condition'].astype('category')
df['storage'] = df['storage'].astype('category')
df['search_engine'] = df['search_engine'].astype('category')
df['channel'] = df['channel'].astype('category')
df['device_type'] = df['device_type'].astype('category')

# El tiempo es mejor manejarlo como tal
df.index = pd.to_datetime(df.index)

# Chequeo
df.info()

In [ ]:
ahorro = (bytes_used - df.memory_usage().sum())
porcentaje = (ahorro/bytes_used) * 100
print("Memoria ahorrada: {:.4f}MB ({:.2f}%)".format(ahorro/1000000,porcentaje))

---
## Análisis de eventos

In [ ]:
event = df['event']
descripcion = pd.read_csv('data/events-desc.csv',index_col='event')
descripcion['value_counts'] = event.value_counts()
with pd.option_context('display.max_colwidth',0):
    display(descripcion)

In [ ]:
orden = event.value_counts().head(7).index
visu = sns.countplot(x='event',data=df,order=orden)
visu.axes.set_title('Cantidad de visitas segun evento')

In [ ]:
df['month'] = df.index.month
df['day'] = df.index.day
for_heatmap = df.pivot_table(index='month',columns='day', values='event', aggfunc='count')
for_heatmap
g = sns.heatmap(for_heatmap,  cmap="OrRd")
g.set_title("Tráfico en sitio según mes y día")
g.set_xlabel("Día")
g.set_ylabel("Mes")

In [ ]:
df['month'] = df.index.month
df['week_day'] = df.index.weekday

for_heatmap = df.pivot_table(index='month',columns='week_day', values='event', aggfunc='count')
for_heatmap.columns = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

g = sns.heatmap(for_heatmap,  cmap="OrRd")
g.set_title("Tráfico en sitio según mes y día de la semana")
g.set_xlabel("Día de la semana")
g.set_ylabel("Mes")

In [ ]:
df['week_day_name'] = df.index.weekday_name
weekday_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
weekday_counts = df.groupby('week_day_name').count()
weekday_counts = weekday_counts.loc[weekday_names]
weekday_counts['event'].plot(kind='bar') #Probando, después se tunea

In [ ]:
df['month_name'] = df['month'].apply(lambda x: calendar.month_abbr[x])
month_name= ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun']
month_counts = df.groupby('month_name').count()
month_counts
month_counts = month_counts.loc[month_name]
month_counts['event'].plot(kind='bar') #Probando, después se tunea

---
## Análisis geográfico

In [ ]:
countries = df['country'].value_counts()
countries = countries.drop('Unknown')
data = countries.head(3)

visu = squarify.plot(data, label=data.index, alpha=.5, color=['green','red','cyan'])
visu.axes.set_title('Países con más visitas')

In [ ]:
# Siempre supimos (asumimos) que Brazil sería el país con mayor tráfico. 
# Este dato no nos deja ver la proporción del resto de los países
countries = df['country'].value_counts()
countries = countries.drop('Unknown')
countries = countries.drop('Brazil')
data = countries.head(7)

visu = squarify.plot(data, label=data.index, alpha=.5, color=['red','cyan','yellow','grey','purple','orange','blue'])
visu.axes.set_title('Países con más visitas, exceptuando Brazil')

**Sacando las longitudes y latitudes de distintas ciudades del mundo, podemos ver que zonas de los 4 países con más trafico son las que más visitas tienen.** 

Con ayuda del módulo geopandas, podemos plotear directamente sobre un mapa
Las bases de datos adicionales fueron sacadas de http://www.geonames.org/

In [ ]:
BR = pd.read_csv('data/BR.csv', low_memory=False, sep='\t')
BR = BR[['name','latitude','longitude']]

# Para que geopandas pueda leer bien las latitudes y longitudes, deben ser pasadas al tipo Point
BR['coordinates'] = list(zip(BR['longitude'],BR['latitude']))
BR['coordinates'] = BR['coordinates'].apply(Point)
BR.head()

In [ ]:
# Se debe hacer un join de los datos que se tienen (nombre de ciudad, cantidad de eventos) y los datos de geonames (nombre de ciudad, punto en el mapa), y esto plotearlo sobre los datos de geopandas (nombre de pais, punto en el mapa mundial)

# Se preparan los datos para el join (inner join de nombre de ciudad (columna name))
ciudades_brazil = df.loc[(df['country'] == 'Brazil')]
data = ciudades_brazil['city'].value_counts()
data = data.drop('Unknown')
data = data.to_frame()
data.reset_index(inplace=True)
data = data.rename(columns={'index':'name','city':'count'})

# Se pasa de un dataframe normal de pandas a uno de geopandas
BRA = gpd.GeoDataFrame(BR, geometry='coordinates')

# Se hace el join (en geopandas: merge) de ambos sets. Siendo que geonames daba mucha más información de la necesaria, duplicando valores por ciudades, se borran los duplicados
data = BRA.merge(data, on='name')
data = data.drop_duplicates('name')

# Se prepara el 'fondo' del gráfico, siendo este nomás el país. Para esto se usan los datos por defecto de geopandas
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
ax = world[world.name=='Brazil'].plot(color='white',edgecolor='black')

visu = data.plot(ax=ax,cmap='OrRd')
visu.axes.set_title('Ciudades de Brazil con más visitas')

In [ ]:
ciudades_argentina = df.loc[(df['country'] == 'Argentina')]
data = ciudades_argentina['city'].value_counts()
data

In [ ]:
# Viendo que la mayoría de los eventos de Argentina vienen de Buenos Aires, hacer el gráfico sobre el mapa sería trivial

---
## Análisis de busquedas

In [ ]:
st = df['search_term'].dropna()
st = st.apply(lambda x: x.lower())
st = st.value_counts().to_frame()
st = st.loc[(st['search_term'] >= 300)]

text = ''
for w,q in zip(st.index,st['search_term']):
    text += ' '.join([w for x in range(q)])

text = ' '.join([s for s in text.split() if len(s)>2])    

wordcloud = WordCloud(width=2000, height=800, margin=0,collocations=False).generate(text)
 
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.margins(x=0, y=0)
plt.show()

## Análisis de páginas estáticas

In [ ]:
df['staticpage'].value_counts()

Podemos ver que la cantidad de visitas a **Customer Service** es mucho mayor a la página de **FAQ**. Para optimizar recursos sería más eficiente intentar de redireccionar el tráfico a FAQ, haciendo más visible los links a la página y de ser necesario mejorándola.

In [ ]:
faq_and_service = df[(df['staticpage'] == 'CustomerService') | (df['staticpage'] == 'FaqEcommerce')]['staticpage']
visu = sns.countplot(faq_and_service)
visu.set(xlabel='Page', ylabel='Visits')
visu.axes.set_title('Cantidad de eventos según staticpage')

## Análisis de New vs Returning

La intención de este  análisis es mostrar que cantidad de usuarios solo ingresan una vez al site.
El primer gráfico no nos aporta información, porque todos los usuarios que volvieron a ingresar, en su primera vez fueron **New**.

In [ ]:
order = df['new_vs_returning'].value_counts().index
visu = sns.countplot(df['new_vs_returning'].dropna(), order=order)
visu.set(xlabel='Tipo de usario', ylabel='Visitas al home')
visu.axes.set_title('Cantidad de eventos según marca')

En este gráfico sí se puede apreciar los valores reales de usuarios que regresan con los que sólo entraron una vez.
Para aumentar la tasa de personas que regresan a ĺa página proponemos aumentar el presupuesto en publicidad y mejorar la experiencia de usuario de la home para que provea al usuario una experiencia más amena.

In [ ]:
gb = df.groupby(['person', 'new_vs_returning']).agg({'new_vs_returning' : 'size'})
gb = gb.unstack(level='new_vs_returning')
gb = gb['new_vs_returning']
returners_count = gb['Returning'].count()
only_once_count = gb['New'].count() - returners_count

objects = ('Regresan', 'Sólo una vez')
y_pos = np.arange(len(objects))
type_users_count = pd.Series([returners_count, only_once_count])
 
plt.bar(y_pos, type_users_count, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Cantidad')
plt.title('Tipos de usuarios')
 
plt.show()

## Análisis de tipos de dispositivos

Cómo la mayoría del tráfico proviene de smartphones y computadoras, debería dedicarse una mayor cantidad de recursos a desarrollar la aplicación para estos dispositivos y no dedicar mucho tiempo y desarrolladores a las aplicaciones para tablets.

In [ ]:
order = df['device_type'].value_counts().index
visu = sns.countplot(df['device_type'].dropna(), order=order)
visu.set(xlabel='Device Type', ylabel='Visits')
visu.axes.set_title('Cantidad de eventos segun marca')

## Análisis de comportamiento de usuarios

## Análisis de cronología de eventos por modelo

In [ ]:
# Qué eventos tenemos asociados a un modelo?
cronologia = df.loc[df['model'].notnull(), ['event','model']]
cronologia['event'].value_counts()

Tomamos como cronología de eventos sobre un modelo:
    1. Visitarlo (`viewed product`)
    1. Mandar a comprarlo (`checkout`)
    1. Efectivamente comprarlo (`conversion`)

In [ ]:
cronologia = cronologia.loc[cronologia['event'] != 'lead'] # Se descartan los eventos 'lead' porque no influyen en el análisis

# Comparemos los modelos por los cuales más se opera segun el evento
vistos = cronologia.loc[cronologia['event']=='viewed product']['model'].value_counts().head(5)
checkouts = cronologia.loc[cronologia['event']=='checkout']['model'].value_counts().head(5)
comprados = cronologia.loc[cronologia['event']=='conversion']['model'].value_counts().head(5)
display(vistos,checkouts,comprados)

In [ ]:
# Visualicemos los modelos más prominentes, haciendo una combinación de los 5 más vistos, los 5 más por comprar  y los 5 más comprados
modelos = set(vistos.index)
modelos.update(checkouts.index)
modelos.update(comprados.index)

cronologia_agrupado = cronologia.groupby('model')['event'].value_counts().unstack('event') # Se agrupa por modelo y se ve por modelos
cronologia_agrupado = cronologia_agrupado[['viewed product', 'checkout', 'conversion']]
modelos_prominentes = cronologia_agrupado.loc[cronologia_agrupado.index.isin(modelos)]
modelos_prominentes

In [ ]:
vistos = modelos_prominentes['viewed product']
checkouts = modelos_prominentes['checkout']
comprados = modelos_prominentes['conversion']

plt.bar(vistos.index, vistos, color=sns.xkcd_rgb["muted blue"],log=True)
plt.bar(checkouts.index, checkouts, color=sns.xkcd_rgb["muted green"],log=True)
plt.bar(comprados.index, comprados, color=sns.xkcd_rgb["muted pink"],log=True)

plt.show()

## ¿Sirve la publicidad de Trocafone para generar más ventas?

Vamos a usar:
-quienes que vienen de una publicidad terminan haciendo una conversion
-cuantos vienen de publicidad y cuantos no
-que pagina es mas efectiva con la publicidad

In [ ]:
publicitados = df.loc[df['campaign_source'].notnull(), ['campaign_source','month']] #(df['event']).isnull().sum() = 0
#plt.bar(publicitados.index, publicitados, color=sns.xkcd_rgb["muted blue"],log=True)
#plt.bar(checkouts.index, checkouts, color=sns.xkcd_rgb["muted green"],log=True)
#plt.bar(comprados.index, comprados, color=sns.xkcd_rgb["muted pink"],log=True)

#plt.show()

publicitados['month'] = publicitados['month'].apply(lambda x: calendar.month_abbr[x])
month_counts = publicitados.groupby('month').count()
month_counts = month_counts.loc[month_name]
month_counts['campaign_source'].plot(kind='bar') #Sabemos que es medio choto pero lo dejamos por las dudas

In [ ]:
ranking = df['campaign_source'].value_counts()
ranking_visu = ranking.head(5)

visu = squarify.plot(ranking_visu, label=ranking_visu.index, alpha=.5, color=['red','cyan','yellow','grey','purple','orange','blue'])
visu.axes.set_title('5 metodos de publicidad más clikeados')


In [ ]:
ranking = ranking.drop('google')
ranking_visu2 = ranking.head(10)

ranking_visu2
visu = squarify.plot(ranking_visu2, label=ranking.index, alpha=.5, color=['red','cyan','yellow','grey','purple','orange','blue'])
visu.axes.set_title('5 metodos de publicidad más clikeados')

---
## Conclusiones (insights)

# TO DO 

- Intro de trocafone
- Info general del dataset
- Columna por columna graficos interesnates
    - Pregunta
    - Grafico
    - Respuesta / Analisis
- Insights generales

Buenos robos:
- https://jilt.com/blog/ecommerce-metrics/
- https://python-graph-gallery.com/
- https://github.com/kevinspasiuk/orgaDatos/blob/master/General.ipynb -> Reloj / Postulaciones y vistas a lo largo del tiempo
- https://github.com/GastonMontes/75.06-Datos-TP1/blob/master/Notebooks/Junto.ipynb -> Sabe analizar
- Apunte cap 2 tiene muy buenos analisis
- https://github.com/lrargerich/7506/blob/master/Notebooks/Analisis%20Exploratorio/Movies%20from%20IMDB.ipynb
- https://github.com/lrargerich/7506/blob/master/Notebooks/Analisis%20Exploratorio/MovieLens.ipynb
- https://github.com/lrargerich/7506/blob/master/Notebooks/Analisis%20Exploratorio/Airplane%20Crashes.ipynb
- https://github.com/lrargerich/7506/blob/master/Notebooks/Analisis%20Exploratorio/Universities%20-%20Times%20ranking.ipynb
- https://github.com/idontdomath/datos-exploratory-data-analysis/blob/master/2018-02/02-kickstarter-projects.ipynb

---
- [ ] ¿Pudieron descubrir features en el campo ‘model’? ¿Cuales fueron?
- [ ] ¿Identificaron patrones o funnels de usuarios que realizan checkouts/conversiones en Trocafone? 
- [ ] ¿Se comportan de forma distinta dependiendo del tipo de dispositivo desde el cual acceden?
- [ ] ¿Se comportan de forma distinta dependiendo del tipo de fuente de tráfico al que pertenecen?
- [ ] ¿Realizaron algún análisis sobre búsquedas que realizan los usuarios y las keywords que utilizan apoyándose en algún tipo de visualización?
- [ ] ¿Realizaron algún análisis de lugar donde se originan las visitas de los usuarios de Trocafone (a nivel país, regiones más importantes o ciudades más importantes) apoyándose en algún tipo de visualización?
- [ ] ¿Pudieron descubrir features jerarquizando información de alguno de los campos (por ejemplo “screen_resolution”)?

---
- [ ] Boxplot, violin, stacked bar, histograma, scatter plot, radar chart
- [ ] Visualicemos los modelos más prominentes por ranking
- [ ] Trafico y compras en mismo grafico de lineas que no se como se llama y es igual (lineal?)
- [ ] Treemap barchart por meses y compras en cada mes. Normalizar el mes de junio!!
- [ ] Grafico de rankings de celular mas viewed pasados los meses
- [ ] Conversiones vs Checkouts 
- [ ] Analisis de mayo. Por que hay tantos datos?
- [ ] Comparacion entre conversiones, checkouts y viewed products. Analizar que porcentaje de futuros compradores se fueron perdiendo
- [ ] Agregar barplot de regiones antes del mapa brazuca
---
- [ ] ¿Tienen todos los ejes su rótulo?
- [ ] ¿Tiene cada visualización un título?
- [ ] ¿Es entendible la visualización sin tener que leer la explicación?
- [ ] ¿El uso del color es adecuado?
- [ ] ¿Hay un exceso o falta de elementos visuales en la visualización elegida?
- [ ] ¿Presenta el grupo un listado de "insights" aprendidos sobre los datos en base al análisis realizado? ¿Es interesante?